In [1]:
using Revise

using PythonCall
using SparseArrays
using Test
using BenchmarkTools
const pf = pyimport("pfapack.pfaffian")
const cpf = pyimport("pfapack.ctypes").pfaffian
const np = pyimport("numpy")
includet("pfaffian.jl")

In [2]:
N = 100

100

In [7]:
# AA = rand(N, N);
# AA = AA .- transpose(AA);

T = [Vector{Float64}, Int64, Float64]

# for i in 1:(size(A, 1)-2)
i = 1
    A0 = AA[i+1:end, i]

    for i in eachindex(T)
        B1 = pyconvert(T[i], pf.householder_real(np.array(A0))[i-1])
        B2 = householder_real(A0)[i]
        display(@test B1 ≈ B2)
    end
# end

@btime householder_real($A0)
# 718.735 ns (1 allocation: 896 bytes)
# @code_warntype householder_real(A0)

t = zeros(size(A0))
@btime householder_real!($t, $A0)
# 639.024 ns (0 allocations: 0 bytes)

Test Passed

Test Passed

Test Passed

  718.735 ns (1 allocation: 896 bytes)


  639.024 ns (0 allocations: 0 bytes)


(2, 3.9907435998460046)

In [18]:
# A = rand(ComplexF64, N, N);
# A = A .- transpose(A);

T = [Vector{ComplexF64}, Int64, ComplexF64]

# for i in 1:(size(A, 1)-2)
i = 1
    A0 = A[i+1:end, i]

    for i in eachindex(T)
        B1 = pyconvert(T[i], pf.householder_complex(np.array(A0))[i-1])
        B2 = householder_complex(A0)[i]
        display(@test B1 ≈ B2)
    end
# end
@btime householder_complex($A0)
# 1.435 μs (1 allocation: 1.77 KiB)

t = zeros(eltype(A0), size(A0))
@btime householder_complex!($t, $A0)
# 1.334 μs (0 allocations: 0 bytes)

Test Passed

Test Passed

Test Passed

  1.435 μs (1 allocation: 1.77 KiB)


  1.334 μs (0 allocations: 0 bytes)


(2, -2.255539398058652 - 5.229035322596745im)

In [10]:
# A = rand(ComplexF64, N, N);
# A = A .- transpose(A);

T = [Matrix{ComplexF64}, Matrix{ComplexF64}]

for i in eachindex(T)
    B1 = pyconvert(T[i], pf.skew_tridiagonalize(np.array(A))[i-1])
    B2 = skew_tridiagonalize(A)[i]
    display(@test B1 ≈ B2)
end

@btime skew_tridiagonalize($A)
# 16.906 ms (1466 allocations: 31.38 MiB)

Test Passed

Test Passed

  16.906 ms (1466 allocations: 31.38 MiB)


(ComplexF64[0.0 + 0.0im 5.069958218743592 - 2.2257060394331103im … 0.0 + 0.0im 0.0 + 0.0im; -5.069958218743592 + 2.2257060394331103im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im -0.023230582659178926 - 0.8105419277097711im; 0.0 + 0.0im 0.0 + 0.0im … 0.023230582659178926 + 0.8105419277097711im 0.0 + 0.0im], ComplexF64[1.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -0.08689633049494194 + 0.0im … -0.01491732506487527 - 0.008049186153870229im 0.08701121934381205 + 0.10779688634982748im; … ; 0.0 + 0.0im -0.01436854091243868 - 0.017623172107990193im … -0.0802133446394058 + 0.07718442715087037im -0.09451000629568132 + 0.03960634734507182im; 0.0 + 0.0im -0.02396823323448408 - 0.06212696059082915im … 0.037809624793957115 + 0.11126466040596358im 0.013889642397126555 - 0.11523663056931109im])

In [11]:
# A = rand(ComplexF64, N, N);
# A = A .- transpose(A);

T = [Matrix{ComplexF64}, Matrix{ComplexF64}, SparseMatrixCSC{Float64, Int64}]

for i in eachindex(T)[1:end-1]
    B1 = pyconvert(T[i], pf.skew_LTL(np.array(A))[i-1])
    B2 = skew_LTL(A)[i]
    display(@test B1 ≈ B2)
end
@test Matrix(skew_LTL(A)[3]) ≈ pyconvert(Matrix{Float64}, pf.skew_LTL(np.array(A))[2].toarray())

@btime skew_LTL($A)
# 8.765 ms (1823 allocations: 20.96 MiB)

Test Passed

Test Passed

  8.765 ms (1823 allocations: 20.96 MiB)


(ComplexF64[0.0 + 0.0im -0.7031149868491631 + 0.8553009622381522im … 0.0 + 0.0im 0.0 + 0.0im; 0.7031149868491631 - 0.8553009622381522im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im -1.380854120387995 + 1.4578346204690675im; 0.0 + 0.0im 0.0 + 0.0im … 1.3808541203879958 - 1.4578346204690678im 0.0 + 0.0im], ComplexF64[1.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im -0.015215060940290783 - 0.4584450618399518im … 1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -0.7109929515519481 - 0.06874005628741457im … 0.4605359916775707 - 0.57061239060303im 1.0 + 0.0im], sparse([1, 88, 20, 90, 45, 51, 19, 9, 80, 41  …  93, 81, 91, 96, 82, 57, 15, 5, 18, 73], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 100, 100))

In [95]:
function test()
    A = rand(ComplexF64, N, N);
    A = A .- transpose(A);

    T = [ComplexF64]

    B1 = pyconvert(T[1], pf.pfaffian_LTL(np.array(A)))
    B2 = pfaffian_LTL(A)
    display(@test B1 ≈ B2)

    @btime pfaffian_LTL($A)
end
test()
# 4.083 ms (914 allocations: 10.68 MiB)
# 2.485 ms (362 allocations: 5.32 MiB)
# 1.191 ms (198 allocations: 388.16 KiB)
# 1.194 ms (150 allocations: 348.66 KiB)
# 1.150 ms (55 allocations: 258.98 KiB)

Test Passed

  1.155 ms (55 allocations: 258.98 KiB)


9.37554880316727e25 + 1.6310201802886743e27im

In [96]:
function test()
    A = rand(Float64, N, N);
    A = A .- transpose(A);

    T = [Float64]

    B1 = pyconvert(T[1], pf.pfaffian_LTL(np.array(A)))
    B2 = pfaffian_LTL(A)
    display(@test B1 ≈ B2)

    @btime pfaffian_LTL($A)
end
test()
# 246.603 μs (55 allocations: 180.14 KiB)

Test Passed

  246.603 μs (55 allocations: 180.14 KiB)


7.680106790846349e17

In [64]:
function test()
    A = rand(ComplexF64, N, N);
    A = A .- transpose(A);

    T = [ComplexF64]

    B1 = pyconvert(T[1], pf.pfaffian_householder(np.array(A)))
    B2 = pfaffian_householder(A)
    display(@test B1 ≈ B2)

    @btime pfaffian_householder($A)
end
test()
# 7.847 ms (699 allocations: 15.61 MiB)
# 7.228 ms (601 allocations: 15.53 MiB)
# 5.967 ms (273 allocations: 5.64 MiB)
# 5.458 ms (108 allocations: 637.03 KiB)
# 5.034 ms (108 allocations: 637.03 KiB)
# 4.762 ms (108 allocations: 637.03 KiB)
# 4.855 ms (106 allocations: 480.73 KiB)
# 3.805 ms (106 allocations: 477.67 KiB)
# 2.789 ms (106 allocations: 477.67 KiB)
# 2.671 ms (106 allocations: 477.67 KiB)

Test Passed

  2.635 ms (104 allocations: 324.44 KiB)


-8.750490181196981e26 - 8.285968266854594e26im

In [66]:
function test()
    A = rand(Float64, N, N);
    A = A .- transpose(A);

    T = [Float64]

    B1 = pyconvert(T[1], pf.pfaffian_householder(np.array(A)))
    B2 = pfaffian_householder(A)
    display(@test B1 ≈ B2)

    @btime pfaffian_householder($A)
end
test()
# 693.684 μs (6 allocations: 158.09 KiB)

Test Passed

  693.684 μs (6 allocations: 158.09 KiB)


7.60136906539197e19

In [80]:
function test()
    A = rand(Float64, N, N);
    A = A .- transpose(A);

    T = [Float64]

    B1 = pyconvert(T[1], pf.pfaffian_schur(np.array(A)))
    B2 = pfaffian_schur(A)
    display(@test B1 ≈ B2)

    @btime pfaffian_schur($A)
end
test()
# 10.489 ms (20 allocations: 432.95 KiB)

Test Passed

  10.571 ms (19 allocations: 432.47 KiB)


4.373876868906428e19

In [101]:
A = rand(ComplexF64, N, N);
A = A .- transpose(A);

T = [ComplexF64]

B1 = pyconvert(T[1], pf.pfaffian(np.array(A), "P"))
B2 = pfaffian(A, method="P")
display(@test B1 ≈ B2)

B3 = det(A)
display(@test B1^2 ≈ B3)
display(@test B2^2 ≈ B3)

@btime pfaffian($A, method=$"P")
# 4.202 ms (923 allocations: 10.75 MiB)
# 1.263 ms (58 allocations: 337.22 KiB)

Test Passed

Test Passed

Test Passed

  1.266 ms (57 allocations: 337.16 KiB)


-1.2384259713631659e27 + 3.8152809529251854e26im

In [102]:
A = rand(Float64, N, N);
A = A .- transpose(A);

T = [Float64]

B1 = pyconvert(T[1], pf.pfaffian(np.array(A), "P"))
B2 = pfaffian(A, method="P")
display(@test B1 ≈ B2)

B3 = det(A)
display(@test B1^2 ≈ B3)
display(@test B2^2 ≈ B3)

@btime pfaffian($AA, method=$"P")
# 1.900 ms (905 allocations: 5.49 MiB)
# 274.525 μs (58 allocations: 258.38 KiB)

Test Passed

Test Passed

Test Passed

  273.853 μs (57 allocations: 258.31 KiB)


4.0939509862088714e18

In [103]:
A = rand(ComplexF64, N, N);
A = A .- transpose(A);

T = [ComplexF64]

B1 = pyconvert(T[1], pf.pfaffian(np.array(A), "H"))
B2 = pfaffian(A, method="H")
display(@test B1 ≈ B2)

B3 = det(A)
display(@test B1^2 ≈ B3)
display(@test B2^2 ≈ B3)

@btime pfaffian($A, method=$"H")
# 2.745 ms (107 allocations: 402.67 KiB)

Test Passed

Test Passed

Test Passed

  2.814 ms (106 allocations: 402.61 KiB)


-1.7831631661032433e27 + 8.991287317221923e26im

In [104]:
A = rand(Float64, N, N);
A = A .- transpose(A);

T = [Float64]

B1 = pyconvert(T[1], pf.pfaffian(np.array(A), "H"))
B2 = pfaffian(A, method="H")
display(@test B1 ≈ B2)

B3 = det(A)
display(@test B1^2 ≈ B3)
display(@test B2^2 ≈ B3)

@btime pfaffian($A, method=$"H")
# 739.114 μs (9 allocations: 236.33 KiB)

Test Passed

Test Passed

Test Passed

  718.669 μs (8 allocations: 236.27 KiB)


-2.2306769194648764e19

# Python

In [67]:
A = rand(ComplexF64, N, N);
A = A .- transpose(A);

Ar = rand(Float64, N, N);
Ar = Ar .- transpose(Ar);

In [17]:
@btime $pf.pfaffian($np.array($A), $"H")
# 7.329 ms (26 allocations: 680 bytes)

  7.329 ms (26 allocations: 680 bytes)


Python: (-6.005140129442108e+25+4.827311008615672e+26j)

In [81]:
@btime $pf.pfaffian($np.array($Ar), $"H")
# 4.772 ms (25 allocations: 664 bytes)

  4.772 ms (25 allocations: 664 bytes)


Python: 8.74278931693956e+19

In [18]:
@btime $pf.pfaffian($np.array($A), $"P")
# 7.243 ms (26 allocations: 680 bytes)

  7.243 ms (26 allocations: 680 bytes)


Python: (-6.005140129442108e+25+4.827311008615672e+26j)

In [82]:
@btime $pf.pfaffian($np.array($Ar), $"P")
# 4.836 ms (25 allocations: 664 bytes)

  4.836 ms (25 allocations: 664 bytes)


Python: 8.74278931693956e+19

# C

In [19]:
@btime cpf($np.array($A), method=$"H")
# 849.166 μs (24 allocations: 624 bytes)

  849.166 μs (24 allocations: 624 bytes)


Python: (-6.00514012944221e+25+4.8273110086157624e+26j)

In [68]:
@btime cpf($np.array($Ar), method=$"H")
# 331.007 μs (23 allocations: 608 bytes)

  331.007 μs (23 allocations: 608 bytes)


Python: 8.742789316939607e+19

In [20]:
@btime cpf($np.array($A), method=$"P")
# 533.480 μs (24 allocations: 624 bytes)

  533.480 μs (24 allocations: 624 bytes)


Python: (-6.005140129442366e+25+4.82731100861568e+26j)

In [84]:
@btime cpf($np.array($Ar), method=$"P")
# 192.462 μs (23 allocations: 608 bytes)

  192.462 μs (23 allocations: 608 bytes)


Python: 8.742789316939645e+19